# Episcopio - Notebook de Procesos ETL

## Tomando el pulso epidemiológico de México

Este notebook documenta y demuestra los procesos ETL (Extract, Transform, Load) de la aplicación Episcopio.
Puede usarse para explorar y ejecutar paso a paso cada uno de los procesos de ingesta, transformación y carga de datos.

### Contenido
1. [Configuración Inicial](#configuracion)
2. [Ingesta de Datos Oficiales](#ingesta-oficial)
3. [Ingesta de Datos Sociales](#ingesta-social)
4. [Normalización y Transformación](#normalizacion)
5. [Análisis y KPIs](#analisis)
6. [Generación de Alertas](#alertas)
7. [Visualizaciones](#visualizaciones)

## 1. Configuración Inicial <a id="configuracion"></a>

Primero importamos las librerías necesarias y configuramos el entorno.

In [ ]:
# Importar librerías estándar
import sys
import os
from datetime import datetime, timedelta
import json

# Agregar directorio raíz al path
sys.path.insert(0, os.path.abspath('.'))

# Importar librerías para análisis de datos
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go

# Configurar visualizaciones
plt.style.use('seaborn-darkgrid')
%matplotlib inline

print("✅ Librerías importadas correctamente")
print(f"Fecha de ejecución: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")

### Configuración de variables de entorno

In [ ]:
# Configurar variables de entorno para la aplicación
os.environ['EP_POSTGRES_HOST'] = os.getenv('EP_POSTGRES_HOST', 'localhost')
os.environ['EP_POSTGRES_USER'] = os.getenv('EP_POSTGRES_USER', 'episcopio')
os.environ['EP_POSTGRES_PASSWORD'] = os.getenv('EP_POSTGRES_PASSWORD', 'changeme')
os.environ['EP_POSTGRES_DATABASE'] = os.getenv('EP_POSTGRES_DATABASE', 'episcopio')
os.environ['EP_POSTGRES_PORT'] = os.getenv('EP_POSTGRES_PORT', '5432')

print("✅ Variables de entorno configuradas")
print(f"   Host: {os.environ['EP_POSTGRES_HOST']}")
print(f"   Database: {os.environ['EP_POSTGRES_DATABASE']}")

## 2. Ingesta de Datos Oficiales <a id="ingesta-oficial"></a>

Este proceso extrae datos de fuentes oficiales como DGE, INEGI, CONACYT y SSA.

In [ ]:
# Importar módulos de ingesta
from ingesta import oficial

print("=" * 60)
print("INGESTA DE DATOS OFICIALES")
print("=" * 60)

### 2.1 Datos de DGE (Dirección General de Epidemiología)

In [ ]:
print("\n📊 Ingesta de datos DGE...")
resultado_dge = oficial.fetch_dge()
print(f"Status: {resultado_dge['status']}")
print(f"Filas procesadas: {resultado_dge['filas_procesadas']}")
print(f"Filas insertadas: {resultado_dge['filas_insertadas']}")

### 2.2 Datos de INEGI

In [ ]:
print("\n📊 Ingesta de datos INEGI...")
resultado_inegi = oficial.fetch_inegi()
print(f"Status: {resultado_inegi['status']}")
print(f"Indicadores actualizados: {resultado_inegi['indicadores_actualizados']}")

### 2.3 Datos de CONACYT COVID-19

In [ ]:
print("\n📊 Ingesta de datos CONACYT...")
resultado_conacyt = oficial.fetch_conacyt_covid()
print(f"Status: {resultado_conacyt['status']}")
print(f"Filas procesadas: {resultado_conacyt['filas_procesadas']}")

### 2.4 Verificación de fuentes

In [ ]:
print("\n🔍 Verificando disponibilidad de fuentes...")
fuentes = oficial.verificar_fuentes()

# Crear DataFrame para mejor visualización
df_fuentes = pd.DataFrame(fuentes)
print(df_fuentes.to_string(index=False))

## 3. Ingesta de Datos Sociales <a id="ingesta-social"></a>

Este proceso extrae datos de redes sociales y fuentes complementarias.

In [ ]:
# Importar módulos de ingesta social
from ingesta import social

print("=" * 60)
print("INGESTA DE DATOS SOCIALES")
print("=" * 60)

### 3.1 Datos de Twitter

In [ ]:
print("\n🐦 Ingesta de Twitter...")
resultado_twitter = social.fetch_twitter()
print(f"Status: {resultado_twitter['status']}")
print(f"Tweets procesados: {resultado_twitter['tweets_procesados']}")

### 3.2 Datos de Facebook

In [ ]:
print("\n👥 Ingesta de Facebook...")
resultado_facebook = social.fetch_facebook()
print(f"Status: {resultado_facebook['status']}")
print(f"Posts procesados: {resultado_facebook['posts_procesados']}")

### 3.3 Datos de Reddit

In [ ]:
print("\n🤖 Ingesta de Reddit...")
resultado_reddit = social.fetch_reddit()
print(f"Status: {resultado_reddit['status']}")
print(f"Posts procesados: {resultado_reddit['posts_procesados']}")

### 3.4 Datos de News API

In [ ]:
print("\n📰 Ingesta de noticias...")
resultado_news = social.fetch_news()
print(f"Status: {resultado_news['status']}")
print(f"Artículos procesados: {resultado_news['articulos_procesados']}")

### 3.5 Clasificación de relevancia y sentimiento

In [ ]:
print("\n🎯 Prueba de clasificación de relevancia...")

# Ejemplos de textos
textos = [
    "Incremento de casos de dengue en Yucatán",
    "Campaña de vacunación contra influenza",
    "El clima está muy agradable hoy",
    "Brote de COVID-19 en hospitales",
    "Receta de cocina tradicional mexicana"
]

resultados = []
for texto in textos:
    es_relevante = social.clasificar_relevancia(texto)
    sentimiento = social.analizar_sentimiento(texto)
    resultados.append({
        'texto': texto[:50] + '...' if len(texto) > 50 else texto,
        'relevante': '✅' if es_relevante else '❌',
        'sentimiento': f"{sentimiento:.2f}"
    })

df_clasificacion = pd.DataFrame(resultados)
print(df_clasificacion.to_string(index=False))

## 4. Normalización y Transformación <a id="normalizacion"></a>

Estos procesos estandarizan los datos crudos para su almacenamiento consistente.

In [ ]:
# Importar módulos ETL
from etl import normaliza

print("=" * 60)
print("NORMALIZACIÓN Y TRANSFORMACIÓN")
print("=" * 60)

### 4.1 Normalización de fechas

In [ ]:
print("\n📅 Normalización de fechas...")

fechas_ejemplos = [
    "15/01/2025",
    "2025-01-15",
    "01/02/2025",
    "2025-12-31"
]

resultados_fechas = []
for fecha in fechas_ejemplos:
    normalizada = normaliza.estandarizar_fecha(fecha)
    resultados_fechas.append({
        'original': fecha,
        'normalizada': normalizada
    })

df_fechas = pd.DataFrame(resultados_fechas)
print(df_fechas.to_string(index=False))

### 4.2 Normalización de códigos geográficos

In [ ]:
print("\n🗺️ Normalización de códigos geográficos...")

# Códigos de entidad
print("\nCódigos de entidad:")
entidades_ejemplo = ['9', '31', '1', '23']
for ent in entidades_ejemplo:
    normalizado = normaliza.normalizar_cve_ent(ent)
    print(f"  {ent:>3} -> {normalizado}")

# Códigos de municipio
print("\nCódigos de municipio:")
municipios_ejemplo = [('31', '050'), ('09', '015'), ('23', '005')]
for ent, mun in municipios_ejemplo:
    normalizado = normaliza.normalizar_cve_mun(ent, mun)
    print(f"  {ent}-{mun} -> {normalizado}")

### 4.3 Normalización de nombres de morbilidades

In [ ]:
print("\n🦠 Normalización de morbilidades...")

morbilidades_ejemplo = [
    "covid",
    "CORONAVIRUS",
    "dengue clasico",
    "gripe",
    "Influenza tipo A"
]

resultados_morb = []
for morb in morbilidades_ejemplo:
    normalizada = normaliza.normalizar_nombre_morbilidad(morb)
    resultados_morb.append({
        'original': morb,
        'normalizada': normalizada
    })

df_morb = pd.DataFrame(resultados_morb)
print(df_morb.to_string(index=False))

### 4.4 Cálculo de semanas ISO

In [ ]:
print("\n📆 Cálculo de semanas ISO...")

# Generar fechas de ejemplo
fechas_iso = pd.date_range(start='2025-01-01', end='2025-01-31', freq='W')

resultados_iso = []
for fecha in fechas_iso:
    fecha_str = fecha.strftime('%Y-%m-%d')
    semana = normaliza.calcular_semana_iso(fecha_str)
    resultados_iso.append({
        'fecha': fecha_str,
        'semana_iso': semana
    })

df_iso = pd.DataFrame(resultados_iso)
print(df_iso.to_string(index=False))

### 4.5 Validación de datos

In [ ]:
print("\n✅ Validación de casos y defunciones...")

casos_prueba = [
    (100, 10),   # Válido
    (50, 5),     # Válido
    (-10, 2),    # Inválido (casos negativos)
    (100, 150),  # Inválido (defunciones > casos)
    (0, 0)       # Válido
]

resultados_val = []
for casos, defunciones in casos_prueba:
    es_valido = normaliza.validar_casos_defunciones(casos, defunciones)
    resultados_val.append({
        'casos': casos,
        'defunciones': defunciones,
        'valido': '✅' if es_valido else '❌'
    })

df_val = pd.DataFrame(resultados_val)
print(df_val.to_string(index=False))

## 5. Análisis y KPIs <a id="analisis"></a>

Generación de datos de muestra y cálculo de indicadores clave.

In [ ]:
print("=" * 60)
print("ANÁLISIS Y KPIS")
print("=" * 60)

# Generar datos de muestra para análisis
np.random.seed(42)
fechas = pd.date_range(start='2024-01-01', end='2025-01-31', freq='D')

# Simular serie temporal de casos con tendencia y estacionalidad
dias = len(fechas)
tendencia = np.linspace(100, 150, dias)
estacionalidad = 20 * np.sin(np.linspace(0, 4*np.pi, dias))
ruido = np.random.normal(0, 10, dias)
casos = (tendencia + estacionalidad + ruido).astype(int)
casos = np.maximum(casos, 0)  # No negativos

# Simular defunciones (proporción de casos)
defunciones = (casos * 0.03 + np.random.normal(0, 1, dias)).astype(int)
defunciones = np.maximum(defunciones, 0)

# Crear DataFrame
df_serie = pd.DataFrame({
    'fecha': fechas,
    'casos': casos,
    'defunciones': defunciones
})

print(f"\n✅ Datos generados: {len(df_serie)} registros")
print(f"   Rango: {df_serie['fecha'].min().date()} a {df_serie['fecha'].max().date()}")
print(f"\nPrimeros registros:")
print(df_serie.head().to_string(index=False))

### 5.1 KPIs básicos

In [ ]:
print("\n📊 KPIs básicos...")

# Calcular KPIs
casos_totales = df_serie['casos'].sum()
defunciones_totales = df_serie['defunciones'].sum()
promedio_casos = df_serie['casos'].mean()
promedio_defunciones = df_serie['defunciones'].mean()
tasa_letalidad = (defunciones_totales / casos_totales * 100) if casos_totales > 0 else 0

print(f"\n  Casos totales: {casos_totales:,}")
print(f"  Defunciones totales: {defunciones_totales:,}")
print(f"  Promedio diario de casos: {promedio_casos:.1f}")
print(f"  Promedio diario de defunciones: {promedio_defunciones:.1f}")
print(f"  Tasa de letalidad: {tasa_letalidad:.2f}%")

### 5.2 Promedios móviles

In [ ]:
print("\n📈 Cálculo de promedios móviles...")

# Calcular promedios móviles
ventanas = [7, 14, 28]
for ventana in ventanas:
    df_serie[f'ma_{ventana}'] = df_serie['casos'].rolling(window=ventana).mean()
    print(f"  MA-{ventana} días calculado")

print("\nÚltimos valores (con promedios móviles):")
print(df_serie[['fecha', 'casos', 'ma_7', 'ma_14', 'ma_28']].tail().to_string(index=False))

### 5.3 Detección de tendencias

In [ ]:
print("\n📉 Análisis de tendencias...")

# Calcular cambios porcentuales
df_serie['cambio_7d'] = df_serie['casos'].pct_change(periods=7) * 100
df_serie['cambio_14d'] = df_serie['casos'].pct_change(periods=14) * 100

# Estadísticas recientes (últimos 30 días)
df_reciente = df_serie.tail(30)
cambio_promedio = df_reciente['cambio_7d'].mean()
cambio_max = df_reciente['cambio_7d'].max()
cambio_min = df_reciente['cambio_7d'].min()

print(f"\n  Cambio promedio (7 días): {cambio_promedio:.2f}%")
print(f"  Cambio máximo (7 días): {cambio_max:.2f}%")
print(f"  Cambio mínimo (7 días): {cambio_min:.2f}%")

if cambio_promedio > 10:
    print("  🔴 Tendencia: CRECIENTE")
elif cambio_promedio < -10:
    print("  🟢 Tendencia: DECRECIENTE")
else:
    print("  🟡 Tendencia: ESTABLE")

## 6. Generación de Alertas <a id="alertas"></a>

Evaluación de reglas de alerta basadas en umbrales y tendencias.

In [ ]:
print("=" * 60)
print("GENERACIÓN DE ALERTAS")
print("=" * 60)

### 6.1 Regla: Incremento súbito

In [ ]:
print("\n⚠️ Evaluando regla: Incremento súbito...")

# Umbral de incremento
umbral_incremento = 20  # 20%

# Obtener últimos valores
ultimo_registro = df_serie.iloc[-1]
casos_actual = ultimo_registro['casos']
ma_14_actual = ultimo_registro['ma_14']

if pd.notna(ma_14_actual) and ma_14_actual > 0:
    delta_porcentaje = ((casos_actual - ma_14_actual) / ma_14_actual) * 100
    
    print(f"\n  Casos actuales: {casos_actual}")
    print(f"  Promedio 14 días: {ma_14_actual:.1f}")
    print(f"  Delta: {delta_porcentaje:.2f}%")
    print(f"  Umbral: {umbral_incremento}%")
    
    if delta_porcentaje > umbral_incremento:
        print(f"\n  🚨 ALERTA ACTIVADA: Incremento súbito detectado")
        print(f"     Los casos actuales superan el promedio en {delta_porcentaje:.2f}%")
    else:
        print(f"\n  ✅ Sin alerta: Incremento dentro del rango esperado")
else:
    print("\n  ⚠️ Datos insuficientes para evaluar")

### 6.2 Regla: Tendencia sostenida

In [ ]:
print("\n⚠️ Evaluando regla: Tendencia sostenida...")

# Evaluar últimos 7 días
ultimos_7 = df_serie.tail(7)
cambios = ultimos_7['cambio_7d'].dropna()

if len(cambios) > 0:
    # Tendencia creciente si la mayoría de cambios son positivos
    cambios_positivos = (cambios > 0).sum()
    porcentaje_positivos = (cambios_positivos / len(cambios)) * 100
    
    print(f"\n  Días con incremento: {cambios_positivos}/{len(cambios)}")
    print(f"  Porcentaje: {porcentaje_positivos:.1f}%")
    
    if porcentaje_positivos >= 70:
        print(f"\n  🚨 ALERTA ACTIVADA: Tendencia creciente sostenida")
        print(f"     {porcentaje_positivos:.1f}% de los días muestran incremento")
    else:
        print(f"\n  ✅ Sin alerta: Tendencia no sostenida")
else:
    print("\n  ⚠️ Datos insuficientes para evaluar")

## 7. Visualizaciones <a id="visualizaciones"></a>

Generación de gráficos para análisis visual de los datos.

In [ ]:
print("=" * 60)
print("VISUALIZACIONES")
print("=" * 60)

### 7.1 Serie temporal de casos

In [ ]:
# Gráfico con matplotlib
fig, ax = plt.subplots(figsize=(14, 6))

ax.plot(df_serie['fecha'], df_serie['casos'], label='Casos diarios', color='#3498db', linewidth=1.5, alpha=0.7)
ax.plot(df_serie['fecha'], df_serie['ma_7'], label='MA 7 días', color='#e74c3c', linewidth=2)
ax.plot(df_serie['fecha'], df_serie['ma_14'], label='MA 14 días', color='#2ecc71', linewidth=2)

ax.set_xlabel('Fecha', fontsize=12)
ax.set_ylabel('Número de casos', fontsize=12)
ax.set_title('Serie Temporal de Casos Confirmados', fontsize=14, fontweight='bold')
ax.legend(loc='upper left')
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("✅ Gráfico de serie temporal generado")

### 7.2 Gráfico interactivo con Plotly

In [ ]:
# Gráfico interactivo
fig = go.Figure()

fig.add_trace(go.Scatter(
    x=df_serie['fecha'],
    y=df_serie['casos'],
    mode='lines',
    name='Casos diarios',
    line=dict(color='#3498db', width=2),
    opacity=0.7
))

fig.add_trace(go.Scatter(
    x=df_serie['fecha'],
    y=df_serie['ma_7'],
    mode='lines',
    name='MA 7 días',
    line=dict(color='#e74c3c', width=3)
))

fig.add_trace(go.Scatter(
    x=df_serie['fecha'],
    y=df_serie['ma_14'],
    mode='lines',
    name='MA 14 días',
    line=dict(color='#2ecc71', width=3)
))

fig.update_layout(
    title='Serie Temporal de Casos Confirmados (Interactivo)',
    xaxis_title='Fecha',
    yaxis_title='Número de casos',
    hovermode='x unified',
    template='plotly_white',
    height=500
)

fig.show()

print("✅ Gráfico interactivo generado")

### 7.3 Análisis de defunciones

In [ ]:
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(14, 10))

# Gráfico 1: Casos vs Defunciones
ax1.plot(df_serie['fecha'], df_serie['casos'], label='Casos', color='#3498db', linewidth=2)
ax1_twin = ax1.twinx()
ax1_twin.plot(df_serie['fecha'], df_serie['defunciones'], label='Defunciones', color='#e74c3c', linewidth=2)

ax1.set_ylabel('Casos', fontsize=11, color='#3498db')
ax1_twin.set_ylabel('Defunciones', fontsize=11, color='#e74c3c')
ax1.set_title('Casos y Defunciones', fontsize=13, fontweight='bold')
ax1.grid(True, alpha=0.3)
ax1.legend(loc='upper left')
ax1_twin.legend(loc='upper right')

# Gráfico 2: Tasa de letalidad
df_serie['tasa_letalidad'] = (df_serie['defunciones'] / df_serie['casos'] * 100).replace([np.inf, -np.inf], np.nan)
df_serie['tasa_letalidad_ma7'] = df_serie['tasa_letalidad'].rolling(window=7).mean()

ax2.plot(df_serie['fecha'], df_serie['tasa_letalidad'], label='Tasa diaria', color='#95a5a6', linewidth=1, alpha=0.5)
ax2.plot(df_serie['fecha'], df_serie['tasa_letalidad_ma7'], label='MA 7 días', color='#e74c3c', linewidth=2)
ax2.set_xlabel('Fecha', fontsize=11)
ax2.set_ylabel('Tasa de letalidad (%)', fontsize=11)
ax2.set_title('Tasa de Letalidad', fontsize=13, fontweight='bold')
ax2.grid(True, alpha=0.3)
ax2.legend()

plt.tight_layout()
plt.show()

print("✅ Gráficos de análisis de defunciones generados")

### 7.4 Distribución y estadísticas

In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))

# Histograma de casos
ax1.hist(df_serie['casos'], bins=30, color='#3498db', alpha=0.7, edgecolor='black')
ax1.axvline(df_serie['casos'].mean(), color='#e74c3c', linestyle='--', linewidth=2, label=f"Media: {df_serie['casos'].mean():.1f}")
ax1.axvline(df_serie['casos'].median(), color='#2ecc71', linestyle='--', linewidth=2, label=f"Mediana: {df_serie['casos'].median():.1f}")
ax1.set_xlabel('Número de casos', fontsize=11)
ax1.set_ylabel('Frecuencia', fontsize=11)
ax1.set_title('Distribución de Casos Diarios', fontsize=13, fontweight='bold')
ax1.legend()
ax1.grid(True, alpha=0.3)

# Box plot de cambios porcentuales
cambios_clean = df_serie['cambio_7d'].dropna()
ax2.boxplot(cambios_clean, vert=True)
ax2.set_ylabel('Cambio porcentual (%)', fontsize=11)
ax2.set_title('Distribución de Cambios (7 días)', fontsize=13, fontweight='bold')
ax2.grid(True, alpha=0.3, axis='y')
ax2.axhline(y=0, color='#e74c3c', linestyle='--', linewidth=1)

plt.tight_layout()
plt.show()

print("✅ Gráficos de distribución generados")

## Resumen del Notebook

Este notebook ha demostrado los procesos ETL completos de Episcopio:

1. ✅ **Ingesta de datos oficiales** - Conectores a DGE, INEGI, CONACYT
2. ✅ **Ingesta de datos sociales** - Twitter, Facebook, Reddit, News
3. ✅ **Normalización** - Fechas, códigos geográficos, morbilidades
4. ✅ **Análisis de KPIs** - Casos, defunciones, tendencias
5. ✅ **Generación de alertas** - Detección de incrementos y tendencias
6. ✅ **Visualizaciones** - Gráficos estáticos e interactivos

### Próximos pasos

- Implementar conexiones reales a APIs
- Integrar con base de datos PostgreSQL
- Agregar modelos de ML para predicción
- Expandir análisis de sentimiento
- Implementar detección de anomalías avanzada

---

**Episcopio** - Tomando el pulso epidemiológico de México 🇲🇽